In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

In [2]:
%matplotlib widget

In [3]:
df = pd.read_csv("data/Bitfinex_BTCUSD_1h.csv", usecols = ['Date', 'Close'])

In [4]:
data = df.iloc[::-1]

In [5]:
data.reset_index(inplace = True, drop = True)

In [6]:
data.head()

,Date,Close
0,2017-10-09 09-AM,4585.7
1,2017-10-09 10-AM,4594.9
2,2017-10-09 11-AM,4603.0
3,2017-10-09 12-PM,4595.5
4,2017-10-09 01-PM,4617.2


In [7]:
data.insert(data.shape[1], 'SMA_7', data['Close'].rolling(window = 7*24, min_periods = 1).mean())
data.insert(data.shape[1], 'SMA_20', data['Close'].rolling(window = 20*24, min_periods = 1).mean())
data.insert(data.shape[1], 'SMA_50', data['Close'].rolling(window = 50*24, min_periods = 1).mean())

In [8]:
data.plot(figsize = (9, 7));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
data.insert(data.shape[1], 'Signal', np.where(data['SMA_7'] > data['SMA_20'], 1.0, 0.0))

In [10]:
data.insert(data.shape[1], 'Position', data['Signal'].diff())

In [11]:
data.head()

,Date,Close,SMA_7,SMA_20,SMA_50,Signal,Position
0,2017-10-09 09-AM,4585.7,4585.700000,4585.700000,4585.700000,0.0,NaN
1,2017-10-09 10-AM,4594.9,4590.300000,4590.300000,4590.300000,0.0,0.0
2,2017-10-09 11-AM,4603.0,4594.533333,4594.533333,4594.533333,0.0,0.0
3,2017-10-09 12-PM,4595.5,4594.775000,4594.775000,4594.775000,0.0,0.0
4,2017-10-09 01-PM,4617.2,4599.260000,4599.260000,4599.260000,0.0,0.0


In [12]:
plt.figure(figsize = (9,7))
# plot close price, short-term and long-term moving averages 
data['Close'].plot(color = 'k', label= 'Close Price') 
data['SMA_7'].plot(color = 'b',label = '7-day SMA') 
data['SMA_20'].plot(color = 'g', label = '20-day SMA')

# plot ‘buy’ signals
plt.plot(data[data['Position'] == 1].index, 
         data['SMA_7'][data['Position'] == 1], 
         '^', markersize = 10, color = 'lime', label = 'buy')

# plot ‘sell’ signals
plt.plot(data[data['Position'] == -1].index, 
         data['SMA_7'][data['Position'] == -1], 
         'v', markersize = 10, color = 'r', label = 'sell')

# plot buy prices
plt.plot(data[data['Position'] == 1].index, 
         data['Close'][data['Position'] == 1], 
         '.', markersize = 10, color = 'lime', label = 'buy price')

# plot sell prices
plt.plot(data[data['Position'] == -1].index, 
         data['Close'][data['Position'] == -1], 
         '.', markersize = 10, color = 'r', label = 'sell price')

# plt.xticks(data.index, data.Date)
plt.ylabel('Price in USD', fontsize = 15 )
plt.xlabel('Date', fontsize = 15 )
plt.title('BTC 7_20_SMA Crossover', fontsize = 20)
plt.legend()
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
class Position():
    def __init__(self, buyPrice, quantity):
        self.buyPrice = buyPrice
        self.quantity = quantity
    def __str__(self):
        return ("buy price: " + str(self.buyPrice) + "\quantity: " + str(self.quantity))

class Portfolio():
    trades = []
    def __init__(self, val):
        self.val = val
        self.pos = Position(0, 0)
    def buyVal(self, price, value):
        self.pos = Position(price, value/price)
        self.val -= value
    def sell(self, price):
        self.val += price * self.pos.quantity
        self.trades.append('buy: '+str(round(self.pos.buyPrice, 2))+'  quantity: '+str(round(self.pos.quantity, 5))+'  sell: '+str(round(price, 2)))

In [14]:
def trade(pf):
    global data
    for i in range(data.shape[0]):
        if(data.iloc[i].Position == 1):
            pf.buyVal(data.iloc[i].Close, pf.val)
        elif(data.iloc[i].Position == -1):
            pf.sell(data.iloc[i].Close)

In [15]:
portfolio = Portfolio(10000)
trade(portfolio)

In [16]:
portfolio.val

34824.075054751214

In [17]:
portfolio.trades

['buy: 5655.1  quantity: 1.76832  sell: 7162.1',
 'buy: 7716.4  quantity: 1.64129  sell: 13477.0',
 'buy: 16671.0  quantity: 1.32684  sell: 13840.0',
 'buy: 10790.0  quantity: 1.70189  sell: 9324.5',
 'buy: 8019.8  quantity: 1.97876  sell: 8518.6',
 'buy: 7587.6  quantity: 2.22156  sell: 6670.8',
 'buy: 6634.8  quantity: 2.23361  sell: 6384.3',
 'buy: 7420.3  quantity: 1.92176  sell: 6992.0',
 'buy: 6686.3  quantity: 2.00962  sell: 6376.1',
 'buy: 6697.1  quantity: 1.9133  sell: 6275.7',
 'buy: 6614.0  quantity: 1.81544  sell: 6490.0',
 'buy: 6450.8  quantity: 1.82647  sell: 5771.8',
 'buy: 3959.3  quantity: 2.6626  sell: 3919.0',
 'buy: 4090.0  quantity: 2.55127  sell: 3583.1',
 'buy: 3698.0  quantity: 2.472  sell: 3923.0',
 'buy: 3949.1  quantity: 2.45567  sell: 7953.92',
 'buy: 8959.6  quantity: 2.18003  sell: 10680.0',
 'buy: 10953.8  quantity: 2.12554  sell: 10397.0',
 'buy: 10891.05  quantity: 2.02911  sell: 10301.0',
 'buy: 8445.7  quantity: 2.47486  sell: 7948.4',
 'buy: 9351.6